In [ ]:
import pandas as pd 
import os
import numpy as np
import itertools
import matplotlib.pyplot as plt
import math
import json
import os
import statistics
from statistics import mean
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!git clone https://github.com/zhangzhenyu13/CSDMetalearningRS.git

Cloning into 'CSDMetalearningRS'...
remote: Enumerating objects: 1042, done.
remote: Total 1042 (delta 0), reused 0 (delta 0), pack-reused 1042
Receiving objects: 100% (1042/1042), 15.70 MiB | 18.08 MiB/s, done.
Resolving deltas: 100% (696/696), done.


In [ ]:
os.chdir("/content/CSDMetalearningRS")

In [ ]:
topk = 20

In [ ]:
from ML_Models.Model_def import *
from DataPrepare.TopcoderDataSet import *
from sklearn import tree,naive_bayes,svm
from sklearn import metrics
import time
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn import ensemble
import warnings
from sklearn.pipeline import Pipeline
from ML_Models.Model_def import ML_model
from sklearn.cluster import KMeans,MiniBatchKMeans
import pickle
import numpy as np
from ML_Models.Model_def import ML_model
from sklearn.cluster import KMeans,MiniBatchKMeans
import pickle
import numpy as np

warnings.filterwarnings("ignore")

In [ ]:
# Define a class NBBayes that inherits from ML_model
class NBBayes(ML_model):

    # Define the constructor of the class
    def __init__(self):
        # Call the constructor of the parent class
        ML_model.__init__(self)

    # Define the predict method of the class that takes input data X and returns predicted output Y
    def predict(self,X):
        # If input data is empty, return an empty numpy array
        if len(X)==0:
            return np.array([],dtype=np.int)
        # If verbose mode is on, print a message that NBBayes is predicting
        if self.verbose>0:
            print(self.name," NBBayes is predicting")
        # Predict the output Y using the trained model and input data X
        Y=self.model.predict_proba(Xtest)
        # Return the predicted output Y
        return Y

    # Define the trainModel method of the class that takes input data Xtrain and labels ytrain for training the model
    def trainModel(self, Xtrain, ytrain):

        print("training NB Model")
        # Get the current time
        t0=time.time()

        # Create a Naive Bayes classifier with Gaussian distribution
        self.model=naive_bayes.GaussianNB()

        # Train the Naive Bayes model using the input data Xtrain and labels ytrain
        self.model.fit(Xtrain, ytrain)

        # Get the time after training the model
        t1=time.time()

        # #measure training result
        # vpredict=self.predict(dataSet.validateX)
        # #print(vpredict)
        # vpredict=np.array(vpredict>self.threshold,dtype=np.int)
        # #print(vpredict)
        # score=metrics.accuracy_score(dataSet.validateLabel,vpredict)
        # cm=metrics.confusion_matrix(dataSet.validateLabel,vpredict)
        # print("model",self.name,"trainning finished in %ds"%(t1-t0),"validate score=%f"%score,"CM=\n",cm)
        
        # Print a message that the training of the Naive Bayes model has finished and show the time taken for training
        print("model",self.name,"training finished in %ds"%(t1-t0))

    # Define the findPath method of the class that returns the path of the trained model
    def findPath(self):
        # Define the path of the trained model as the name of the model with a ".pkl" extension
        modelpath=self.name+".pkl"
        # Return the model path
        return modelpath

In [ ]:
class DecsionTree(ML_model):
    # Initialize decision tree parameters
    def initParameters(self):
        self.params={
            'criterion':"gini",
            'splitter':"best",
            'max_depth':5,
            'min_samples_split':2,
            'min_samples_leaf':1,
            'max_features':'auto',
        }
    
    # Initialize decision tree model and parameters
    def __init__(self):
        ML_model.__init__(self)
        self.initParameters()
    
    # Predict with decision tree model
    def predict(self,X):
        if len(X) == 0:
            return np.array([], dtype=np.int)
        if self.verbose > 0:
            print(self.name," C4.5 is predicting")
        
        Y = self.model.predict_proba(X)
        return Y
    
    # Update decision tree parameters
    def updateParameters(self, paras):
        for k in paras:
            self.params[k] = paras[k]
    
    # Search for best decision tree parameters using grid search
    def searchParameters(self, Xtrain, ytrain):
        print("searching for best parameters")
        try:
            # Define parameter grid for grid search
            selParas=[
                {'criterion':["gini",'entropy']},
                {'splitter':["best",'random']},
                {'max_depth':[i for i in range(3,10)]},
                {'min_samples_split':[i for i in range(2,10)]},
                {'min_samples_leaf':[i for i in range(1,10)]},
                {'max_features':[None,'sqrt','log2']},
            ]

            # Perform grid search for each parameter set in selParas and update parameters accordingly
            for i in range(len(selParas)):
                para=selParas[i]
                model=tree.DecisionTreeClassifier(**self.params)
                gsearch=GridSearchCV(model,para,scoring='accuracy')
                gsearch.fit(Xtrain,ytrain)
                print("best para",gsearch.best_params_)
                self.updateParameters(gsearch.best_params_)

            # Create decision tree model with updated parameters
            self.model=tree.DecisionTreeClassifier(**self.params)
        except:
            self.model=tree.DecisionTreeClassifier()
    
    # Train decision tree model with training data
    def trainModel(self, Xtrain, ytrain):
        print("training")
        t0=time.time()

        # Search for best parameters using grid search
        self.searchParameters(Xtrain, ytrain)

        # Fit decision tree model to training data with updated parameters
        self.model.fit(Xtrain, ytrain)

    # Find file path for decision tree model
    def findPath(self):
        modelpath=self.name+".pkl"
        return modelpath


In [ ]:
# Creating Random Forest Class
class RandForest(ML_model):
    
    # Initializing Default Parameters
    def initParameters(self):
        self.params={
            "n_estimators":10,
            "criterion":"gini",
            "max_depth":None,
            "min_samples_split":2,
            "min_samples_leaf":1,
            "min_weight_fraction_leaf":0.,
            "max_features":"auto",
            "max_leaf_nodes":None,
            "min_impurity_decrease":0.,
            "n_jobs":-1
        }
    
    # Constructor Method
    def __init__(self):
        ML_model.__init__(self) # Inheriting from Parent Class
        self.initParameters()   # Initializing Parameters
    
    # Predict Method to Predict Classes
    def predict(self,X):
        if  len(X)==0:
            return np.array([],dtype=np.int)
        if self.verbose>0:
            print(self.name," RF is predicting")

        Y=self.model.predict_proba(X)
        return Y
    
    # Method to Update Model Parameters
    def updateParameters(self,paras):
        for k in paras:
            self.params[k]=paras[k]
    
    # Method to Search for Best Parameters
    def searchParameters(self,Xtrain, ytrain):
        print("searching for best parameters")
        
        try:
            # Parameters to be Searched
            selParas=[
              {"n_estimators":[i for i in range(10,101,10)]},
              {"criterion":["gini","entropy"]},
              {"max_depth":[i for i in range(5,12)]},
              {"min_samples_split":[2,5,7,10]},
              {"min_samples_leaf":[1,2,3,4,5]},
              {"max_features":["log2","sqrt"]},
              {"min_impurity_decrease":[i/100.0 for i in range(0,100,5)]}
            ]

            # Iterating through All Parameters
            for i in range(len(selParas)):
                para=selParas[i]
                model=ensemble.RandomForestClassifier(**self.params) # Creating Random Forest Model
                gsearch=GridSearchCV(model,para,scoring=metrics.make_scorer(metrics.accuracy_score)) # Grid Search Cross Validation
                gsearch.fit(Xtrain, ytrain) # Fitting Grid Search to Data
                print("best para",gsearch.best_params_) # Printing Best Parameters found
                self.updateParameters(gsearch.best_params_) # Updating Best Parameters

            self.model=ensemble.RandomForestClassifier(**self.params) # Creating Random Forest with Updated Parameters
        except:
            self.model=ensemble.RandomForestClassifier(**self.params) # Creating Random Forest with Default Parameters
    
    # Training Method
    def trainModel(self,Xtrain, ytrain):
        print("training")
        t0=time.time()

        self.searchParameters(Xtrain, ytrain) # Searching for Best Parameters

        self.model.fit(Xtrain, ytrain) # Fitting Data to Model
        # t1=time.time()

        # #measure training result
        # vpredict=self.predict(dataSet.validateX)
        # #print(vpredict)
        # vpredict=np.array(vpredict>self.threshold,dtype=np.int)
        # #print(vpredict)
        # score=metrics.accuracy_score(dataSet.validateLabel,vpredict)
        # cm=metrics.confusion_matrix(dataSet.validateLabel,vpredict)
        # print("model",self.name,"trainning finished in %ds"%(t1-t0),"validate score=%f"%score,"CM=\n",cm)

    def findPath(self):
        modelpath= self.name+".pkl"
        return modelpath

In [ ]:
class ClusteringModel(ML_model):
    def __init__(self):
        # Call the constructor of the base class
        ML_model.__init__(self)

    def trainCluster(self,X,n_clusters,minibatch=False):
        # Train a KMeans clustering model on the data X with n_clusters clusters
        # If minibatch is True, use the MiniBatchKMeans algorithm instead of KMeans
        if minibatch:
            km = MiniBatchKMeans(n_clusters=n_clusters, verbose=False)
        else:
            km = KMeans(n_clusters=n_clusters,verbose=False)

        km.fit(X) # Fit the model to the data
        self.model=km # Save the trained model

    def predictCluster(self,X):
        # Predict the cluster assignments for the data X using the trained model
        print(self.name+" is clustering %d tasks"%len(X))
        return  self.model.predict(X)

    def findPath(self):
        # Return the path to save the model
        modelPath= self.name+".pkl"
        return modelPath

In [ ]:
class CBCModel:
    def __init__(self,learners,clustering):
        self.models={} # dictionary to store learners
        for i in range(len(learners)):
            self.models[i]=learners[i] # assign learner to a cluster
        self.clustering=clustering # clustering model used to predict clusters

    def predict(self,X,clustered=False):
        if clustered:
            cluster_no=self.clustering.predict(X) # predict cluster number using clustering model
            Y=self.models[cluster_no].predict(X) # use classifier for predicted cluster to predict outcome
            return Y

        Y=self.models[0].predict(X) # if not clustered, use first classifier to predict outcome
        return Y

def buildCBC(clusters=0, classifier_choice=0):
    if classifier_choice == 0:
      classifier=NBBayes
    else:
      classifier=DecsionTree
    learners=[]
    if clusters>0:
        for i in range(clusters):
            learner=classifier()
            if classifier_choice == 0:
              learner.name="NB-classifier_{}".format(i) # set name of learner based on its index
            elif classifier_choice == 1:
              classifier.name= "RandomForest_{}".format(i)
            else:
              learner.name = "DecisionTree_{}".format(i)
            learner.loadModel()
            learners.append(learner)
    else:
        classifier=classifier()
        if classifier_choice == 0:
            classifier.name= "NB-classifier_{}".format(0) # set name of classifier as only one is used
        elif classifier_choice == 1:
            classifier.name= "RandomForest_{}".format(0)
        else:
            classifier.name = "DecisionTree_{}".format(0)
        classifier.loadModel()
        learners.append(classifier)

    clustering=ClusteringModel()

    return CBCModel(learners,clustering)

def train(Xtrain, ytrain, cluster_num, classifier_num=0):
    # build classifier for given cluster number
    if classifier_num == 0:
      model=NBBayes()
      model.name="NB-classifier_{}".format(cluster_num)
    elif classifier_num == 1:
      model=RandForest()
      model.name="RandomForest_{}".format(cluster_num)
    else:
      model=DecsionTree()
      model.name="DecisionTree_{}".format(cluster_num)
    #train model
    model.trainModel(Xtrain, ytrain)
    model.saveModel()
    return




In [ ]:
all_data = pd.read_csv('/content/Full_dataset_1.csv')

In [ ]:
all_data.columns

Index(['challengeId', 'task_created_date', 'handle', 'Activity Type',
       'activechallenges', 'completedchallenges', 'totalChallengsJoined',
       'totalWins', 'Success_rate', 'task_recency (in days)', 'member_skills',
       'member_description', 'task_name', 'task_languages', 'task_description',
       'Cosine_similarity_score_task_titles_current_past',
       'Cosine_similarity_score_task_descriptions_current_past',
       'Cosine_similarity_score_task_skills_current_past',
       'Cosine_similarity_descriptions', 'Cosine_similarity_skills',
       'number_interactions', 'CP_score'],
      dtype='object')

In [ ]:
all_data.dropna(inplace=True)
all_data.sort_values("task_created_date", ascending=True, inplace=True)

In [ ]:
all_data.columns

Index(['challengeId', 'task_created_date', 'handle', 'Activity Type',
       'activechallenges', 'completedchallenges', 'totalChallengsJoined',
       'totalWins', 'Success_rate', 'task_recency (in days)', 'member_skills',
       'member_description', 'task_name', 'task_languages', 'task_description',
       'Cosine_similarity_score_task_titles_current_past',
       'Cosine_similarity_score_task_descriptions_current_past',
       'Cosine_similarity_score_task_skills_current_past',
       'Cosine_similarity_descriptions', 'Cosine_similarity_skills',
       'number_interactions', 'CP_score'],
      dtype='object')

In [ ]:
all_data['task_description']

3893    http://apps.topcoder.com/wiki/display/docs/Mod...
3892    http://apps.topcoder.com/wiki/display/docs/Mod...
648     Main DARPA CS-STEM page|DARPA CS-STEM active c...
5440    Check the requirements specification here: Req...
5441    Check the requirements specification here: Req...
                              ...                        
3098    ![Odyssey](https://drive.google.com/uc?export=...
3097    ![Odyssey](https://drive.google.com/uc?export=...
3096    ![Odyssey](https://drive.google.com/uc?export=...
3099    ![Odyssey](https://drive.google.com/uc?export=...
3095    ![Odyssey](https://drive.google.com/uc?export=...
Name: task_description, Length: 15200, dtype: object

In [ ]:
all_data.columns

Index(['challengeId', 'task_created_date', 'handle', 'Activity Type',
       'activechallenges', 'completedchallenges', 'totalChallengsJoined',
       'totalWins', 'Success_rate', 'task_recency (in days)', 'member_skills',
       'member_description', 'task_name', 'task_languages', 'task_description',
       'Cosine_similarity_score_task_titles_current_past',
       'Cosine_similarity_score_task_descriptions_current_past',
       'Cosine_similarity_score_task_skills_current_past',
       'Cosine_similarity_descriptions', 'Cosine_similarity_skills',
       'number_interactions', 'CP_score'],
      dtype='object')

In [ ]:
features = ['Activity Type',
       'activechallenges', 'completedchallenges', 'totalChallengsJoined',
       'totalWins', 'Success_rate (%)', 'task_recency (in days)',
       'number_interactions', 'sum_weights_edges', 'CP_score']

categorical = ['task_name', 'task_languages',
       'task_description']

In [ ]:
all_data.shape

(15200, 22)

In [ ]:
all_data['challengeId'].unique()

array(['3dc492e6-fb1b-44ab-9e4a-2d94d1107269',
       '0969087a-d80e-42af-9869-0098959e1749',
       '59266f47-08b2-482b-b1bb-dedee9f576f7', ...,
       'b04553c3-c4b3-4110-a576-3c330b923885',
       '5488a029-f1e7-4473-8813-b6da76ac1765',
       '30150750-d9c9-48a6-89c2-2006c7e5fa2f'], dtype=object)

In [ ]:
all_data.columns

Index(['challengeId', 'task_created_date', 'handle', 'Activity Type',
       'activechallenges', 'completedchallenges', 'totalChallengsJoined',
       'totalWins', 'Success_rate', 'task_recency (in days)', 'member_skills',
       'member_description', 'task_name', 'task_languages', 'task_description',
       'Cosine_similarity_score_task_titles_current_past',
       'Cosine_similarity_score_task_descriptions_current_past',
       'Cosine_similarity_score_task_skills_current_past',
       'Cosine_similarity_descriptions', 'Cosine_similarity_skills',
       'number_interactions', 'CP_score'],
      dtype='object')

In [ ]:
# considering the numerical columns
features = all_data.columns[all_data.dtypes != object]

In [ ]:
features

Index(['activechallenges', 'completedchallenges', 'totalChallengsJoined',
       'totalWins', 'Success_rate', 'task_recency (in days)',
       'Cosine_similarity_score_task_titles_current_past',
       'Cosine_similarity_score_task_descriptions_current_past',
       'Cosine_similarity_score_task_skills_current_past',
       'Cosine_similarity_descriptions', 'Cosine_similarity_skills',
       'number_interactions', 'CP_score'],
      dtype='object')

In [ ]:
overall_metrics = []
last_x_tasks = 10
unique_task_ids = all_data['challengeId'].unique()

for k, STUDIED_TASK in enumerate(unique_task_ids[1:]):
    try:
      if k >= last_x_tasks:
        metrics = []
        TASK_DF = all_data[all_data['challengeId'].isin(unique_task_ids[max(0, k-last_x_tasks+1):k+1])]
        unique_devs = TASK_DF['handle'].unique()
        # take means for devs whose name occurs more than once in the set
        TASK_DF = TASK_DF.groupby('handle').mean().reset_index()
        DF = all_data[all_data['challengeId'] ==  STUDIED_TASK].reset_index(drop=True)
        DF = DF[DF['handle'].isin(unique_devs)].reset_index(drop=True)
        DF = DF.groupby('handle').mean().reset_index()
        unique_handles = DF['handle'].unique()
        
        lbl = LabelEncoder()
        TASK_DF['handle'] = lbl.fit_transform(TASK_DF['handle'].values.reshape(-1,1))
        DF['handle'] = lbl.transform(DF['handle'].values.reshape(-1,1))

        # train on TASK_DF with handle as label and 
        Xtrain = TASK_DF.loc[:, TASK_DF.columns != 'handle']
        ytrain = TASK_DF['handle']


        train(Xtrain[features], ytrain, 0, 0)
        cbc_model = buildCBC(0, 0) 
        Xtest = DF.loc[:, DF.columns != 'handle']
        ytest = DF['handle']
        ypred = cbc_model.predict(Xtest[features])
        metrics = []

        for i in range(ypred.shape[0]):
          df = pd.DataFrame(np.arange(1, len(unique_devs)+1), columns=['devs'])
          df['probability'] = ypred[i,:]

          df.sort_values(by='probability', inplace=True, ascending=False)
          df = df.head(topk)
          rankings = df['devs'].values.tolist()
          precision = 0
          recall = 0
          RR = 0
          if ytest.values[0] in rankings:
            precision = 1/topk
            recall = 1
            RR = 1/(rankings.index(ytest.values[0])+1) if ytest.values[0] in rankings else 0.001
          metrics.append({"precision": precision, "recall":recall, "RR": RR})
        metrics = pd.DataFrame(metrics)
        print("metrics for the task: ", metrics.mean().to_dict())
        overall_metrics.append(metrics.mean().to_dict())

        
    except Exception as ex:
      print("Exception {}".format(ex))
      continue
        
metrics = pd.DataFrame(overall_metrics)
metrics.to_csv("task_details.csv", index=False)
print(metrics.mean())


Streaming output truncated to the last 5000 lines.
training NB Model
model NB-classifier_0 training finished in 0s
NB-classifier_0  NBBayes is predicting
metrics for the task:  {'precision': 0.0, 'recall': 0.0, 'RR': 0.0}
training NB Model
model NB-classifier_0 training finished in 0s
NB-classifier_0  NBBayes is predicting
metrics for the task:  {'precision': 0.0, 'recall': 0.0, 'RR': 0.0}
training NB Model
model NB-classifier_0 training finished in 0s
NB-classifier_0  NBBayes is predicting
metrics for the task:  {'precision': 0.0, 'recall': 0.0, 'RR': 0.0}
training NB Model
model NB-classifier_0 training finished in 0s
NB-classifier_0  NBBayes is predicting
metrics for the task:  {'precision': 0.0, 'recall': 0.0, 'RR': 0.0}
training NB Model
model NB-classifier_0 training finished in 0s
NB-classifier_0  NBBayes is predicting
metrics for the task:  {'precision': 0.0, 'recall': 0.0, 'RR': 0.0}
training NB Model
model NB-classifier_0 training finished in 0s
NB-classifier_0  NBBayes is pr